## Death Count and Dose Vaccinated Ralation

This is a graph that shows the accumulation of deaths and vaccinated counts.

The statistic data are collected from Public Health England (PHE), and the relation between the factors are indicated as such.

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

# embed matplotlib output and make figures larger
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [2]:
# In[] ACCESSING DATA
def access_api():
    filters = ['areaType=overview']
    structure = {
        "date": "date",
        "Deaths": "cumDeaths28DaysByPublishDate", 
        "FirstDose":"cumPeopleVaccinatedFirstDoseByVaccinationDate",
        "SecondDose":"cumPeopleVaccinatedSecondDoseByVaccinationDate"
    }
    api = Cov19API(filters=filters, structure=structure)
    deathVSdose=api.get_json()
    with open("DvD.json", "wt") as OUTF:
        json.dump(deathVSdose, OUTF)
    return OUTF

In [3]:
# In[] WRANGLE DATA

def wrangle_data(rawdata):
    
    # load file and sort date
    with open("DvD.json", "rt") as INFILE:
        data=json.load(INFILE)
    datalist=data['data']
    dates=[dictionary['date'] for dictionary in datalist]
    dates.sort()

    # convert to pandas datetime object
    def parse_date(datestring):
        return pd.to_datetime(datestring, format="%Y-%m-%d")
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])

    # build DF with date index
    index=pd.date_range(startdate, enddate, freq='D')
    DvDdf=pd.DataFrame(index=index, columns=['Deaths', 'FirstDose', 'SecondDose'])

    # replace NULL with 0
    for entry in datalist:
        date=parse_date(entry['date'])
        for column in ['Deaths', 'FirstDose', 'SecondDose']:
            if pd.isna(DvDdf.loc[date, column]):
                value= int(entry[column]) if entry[column]!=None else 0
                DvDdf.loc[date, column]=value
    # with missing dates
    DvDdf.fillna(0, inplace=True)
    
    return DvDdf

In [4]:
# In[] INTERACTIVE

# interactive window
info=wdg.SelectMultiple(
    options=['Deaths', 'FirstDose', 'SecondDose'],
    rows=3,
    disabled=True,
    description='Information:'
)
scale=wdg.RadioButtons(
    options=['linear', 'log'],
    disabled=True,
    description='Scale:'
)

# refresh data button
def access_api_callback(button):
    apibutton.icon='refresh'
    apibutton.description='REFRESH DATA'
    info.disabled=False
    scale.disabled=False
    apidata=access_api()
    global df
    df=wrangle_data(apidata)
    graph(info,scale)

apibutton=wdg.Button(
    description='LOAD DATA',
    button_style='info',
    tooltip='Click to download current Public Health England data',
    icon='download'
)

apibutton.on_click(access_api_callback)

In [5]:
# In[] Display

# display instruction when initialising
def graph(gcols,gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    choice=len(gcols)
    if choice>0:
        df[list(gcols)].plot(logy=logscale) # for log scale
        plt.show() # update graph when interact again
    else:
        print("Load the data and choose the information you want to know")
        print("(CTRL-Click to select more than one category)")

# capture output in widget graph
output=wdg.interactive_output(graph, {'gcols': info, 'gscale': scale})
# create columns
lcol=wdg.VBox([info, scale])
rcol=wdg.VBox([apibutton, output])
# put side by side
form=wdg.HBox([lcol,rcol])
# Print all
display(form)

Button(button_style='info', description='LOAD DATA', icon='download', style=ButtonStyle(), tooltip='Click to d…